# Données issues du fichier des immobilisations corporelles
***

Le recoupemement avec les liasses ne concerne que les sociétés
relevant du régime réel normal (liasses 2033 exclues)<br>

<font color='red'>Le fichier des immobilisations Excel sous format .xlsx doit comprendre au moins les champs suivants : <br>
-> BRUT (titre de la colonne) : valeur brute HT de chaque immobilisation <br>
-> PCG (titre de la colonne) : compte comptable affecté à chaque immobilisation</font>

In [1]:
# Installation du package directement à partir de Jupyter
%pip install xlsxwriter

In [2]:
import openpyxl
import pandas as pd
import numpy as np
import xlsxwriter

### Récupération du fichier des immobilisations

<font color='green'> Récupération du chemin fichier des immobilisations : <br>
==> Copier / coller le chemin d'accès au fichier d'immobilisation concerné</font>

In [4]:
my_directory = input("Chemin où se trouve le fichier des immobilisations : ")

<font color='green'> Récupération du fichier des immobilisations : <br>
==> Saisir uniquement le nom du fichier sans rajouter l'extension '.xlsx'</font>

In [ ]:
my_file = input("Fichier des immobilisations sous format .xlsx : ")

In [ ]:
# Conversion du fichier Excel en dataframe
assets_df = pd.read_excel(f'{my_directory}\{my_file}.xlsx')

In [5]:
# Conversion du champ PCG en type 'str' 
assets_df['PCG'] = assets_df['PCG'].astype('str')

### Recoupement avec les données comptables (balance)

In [6]:
"Regroupement par compte pour recouper avec la comptabilité"

# TCD : valeur brute par compte comptable
balance_df = assets_df.groupby('PCG')['BRUT'].sum().reset_index()

# Colonne renommée
balance_df.rename(columns={"BRUT": "BRUT_FICHIER_IMMOS"}, inplace=True)

# Insertion de nouvelles colonnes
balance_df.insert(loc=balance_df.shape[1], column="BRUT_BALANCE", value='')
balance_df.insert(loc=balance_df.shape[1], column="ECART", value='')
balance_df.insert(loc=balance_df.shape[1], column="OBSERVATIONS", value='')

In [7]:
"Information que le compte répertorié n'est pas un compte du PCG"

# Si le compte commence par 216, 217 ou 219 : compte non répertorié
balance_df.loc[balance_df['PCG']
           .astype('str').str.contains('^216|^217|^219'), 
           'OBSERVATIONS'] = "Compte non répertorié par le PCG"

# Arrondis de la colonne BRUT_FICHIER_IMMOS
balance_df['BRUT_FICHIER_IMMOS'] = balance_df[
    'BRUT_FICHIER_IMMOS'].round(decimals=0)

# Champs à afficher
balance_df = balance_df[[
    "PCG", "BRUT_FICHIER_IMMOS", "BRUT_BALANCE", "ECART", "OBSERVATIONS"]]

In [9]:
# Insertion d'une nouvelle DF pour ajouter le montant total des immobilisations
sum_df = pd.DataFrame({
    'PCG': ['TOTAL'], 
    'BRUT_FICHIER_IMMOS':[balance_df['BRUT_FICHIER_IMMOS'].sum()],
    'BRUT_BALANCE':[''], 'ECART':[''], 'OBSERVATIONS':['']})

In [10]:
# Concaténation des 2 DF ci-avant
balance_df = pd.concat([balance_df, sum_df], axis=0)

### Recoupement avec les liasses fiscales (régime RN)

In [11]:
"Regroupement par poste de la liasse 2054"

# Terrains
fields = int(round(balance_df.loc[
    balance_df['PCG'].astype('str').str.contains('^211|^212')]
            .agg({'BRUT_FICHIER_IMMOS':'sum'})[0], 0))

# Constructions sur sol propre
construct1 = int(round(balance_df.loc[
    ((balance_df['PCG'].astype('str').str.contains('^213')) &
     (~balance_df['PCG'].astype('str').str.contains('^2135')))]
            .agg({'BRUT_FICHIER_IMMOS':'sum'})[0], 0))

# Constructions sur sol d'autrui
construct2 = int(round(balance_df.loc[
    balance_df['PCG'].astype('str').str.contains('^214')]
            .agg({'BRUT_FICHIER_IMMOS':'sum'})[0], 0))

# Constructions Agencements
construct3 = int(round(balance_df.loc[
    balance_df['PCG'].astype('str').str.contains('^2135')]
            .agg({'BRUT_FICHIER_IMMOS':'sum'})[0], 0))

# ITMOI
material = int(round(balance_df.loc[
    balance_df['PCG'].astype('str').str.contains('^215')]
            .agg({'BRUT_FICHIER_IMMOS':'sum'})[0], 0))

# Autres immobilisations agencements
others = int(round(balance_df.loc[
    balance_df['PCG'].astype('str').str.contains('^2181|^2188')]
            .agg({'BRUT_FICHIER_IMMOS':'sum'})[0], 0))

# Matériel de transport
transport = int(round(balance_df.loc[
    balance_df['PCG'].astype('str').str.contains('^2182')]
            .agg({'BRUT_FICHIER_IMMOS':'sum'})[0], 0))

# Matériel de bureau, informatique, mobilier
computer = int(round(balance_df.loc[
    balance_df['PCG'].astype('str').str.contains(
        '^2183|^2184')]
            .agg({'BRUT_FICHIER_IMMOS':'sum'})[0], 0))

# Emballages récupérables et divers
packaging = int(round(balance_df.loc[
    balance_df['PCG'].astype('str').str.contains(
        '^2185|^2186')]
            .agg({'BRUT_FICHIER_IMMOS':'sum'})[0], 0))


In [12]:
# Nouvelle DF pour effectuer le recoupement avec la liasse 2054
financial_df = pd.DataFrame({
    "POSTE":[
        "Terrains", "Constructions sur sol propre", 
        "Constructions sur sol d'autrui", "Constructions Agencements",
        "ITMOI", "Autres immobilisations Agencements",
        "Matériel de transport", "Matériel Bureau, Informatique, Mobilier",
        "Emballages récupérables et divers"],
    "BRUT_FICHIER_IMMOS":[
        fields, construct1, construct2, construct3, material, 
        others, transport, computer, packaging],
    "BRUT_LIASSE_2054":[
        '', '', '', '', '', '', '', '', ''],
    "ECART":['', '', '', '', '', '', '', '', ''],
    "OBSERVATIONS":['', '', '', '', '', '', '', '', '']}) 

In [13]:
# Insertion d'une nouvelle DF pour ajouter le montant total des immobilisations
sum2_df = pd.DataFrame({
    'POSTE': ['TOTAL'], 
    'BRUT_FICHIER_IMMOS':[financial_df['BRUT_FICHIER_IMMOS'].sum()],
    'BRUT_LIASSE_2054':[''], 'ECART':[''], 'OBSERVATIONS':['']})

In [14]:
# Concaténation des 2 DF ci-avant
financial_df = pd.concat([financial_df, sum2_df], axis=0)

### Export des données ci-avant sous format Excel

<font color='green'>Copier / coller le chemin d'export du fichier Excel</font>

In [15]:
export_file = input("Chemin d'export du fichier Excel : ")

In [16]:
# Exportation des données et création des onglets
excelwriter = pd.ExcelWriter(
    f'{export_file}\Recoupement_Immos.xlsx', engine='xlsxwriter')
balance_df.to_excel(
            excelwriter, sheet_name='Recoupement_Compta', index=False)
worksheet_balance = excelwriter.sheets['Recoupement_Compta']
financial_df.to_excel(
            excelwriter, sheet_name='Recoupement_Liasses', index=False)
worksheet_financial = excelwriter.sheets['Recoupement_Liasses']
workbook = excelwriter.book

In [17]:
# Mise en forme des cellules et alignement des cellules
text_format = workbook.add_format()
num_format = workbook.add_format({'num_format': '#,##0 €'})

# Alignement centré de toutes les cellules
text_format.set_align('center')
num_format.set_align('center')

In [18]:
# Bordure des cellules
border_excel = workbook.add_format(
    {'bottom':1, 'top':1, 'left':1, 'right':1})
worksheet_balance.conditional_format(
    xlsxwriter.utility.xl_range(
        0, 0, len(balance_df), balance_df.shape[1]-1), 
    {'type': 'no_errors', 'format': border_excel})   
worksheet_financial.conditional_format(
    xlsxwriter.utility.xl_range(
        0, 0, len(financial_df), financial_df.shape[1]-1), 
    {'type': 'no_errors', 'format': border_excel}) 

0

In [19]:
# Largeur et format des cellules
for column in balance_df:
    column_length = max(balance_df[column]
                        .astype(str).map(len).max(), len(column))
    col_idx = (balance_df.columns.get_loc(column))
    excelwriter.sheets['Recoupement_Compta'].set_column(
        col_idx, col_idx, column_length, text_format)
excelwriter.sheets['Recoupement_Compta'].set_column('A:A', 20, text_format)
excelwriter.sheets['Recoupement_Compta'].set_column('B:B', 25, num_format)
excelwriter.sheets['Recoupement_Compta'].set_column('C:D', 25, text_format)
excelwriter.sheets['Recoupement_Compta'].set_column('E:E', 60, text_format)
for column in financial_df:
    column_length = max(financial_df[column]
                        .astype(str).map(len).max(), len(column))
    col_idx = (financial_df.columns.get_loc(column))
    excelwriter.sheets['Recoupement_Liasses'].set_column(
        col_idx, col_idx, column_length, text_format)
excelwriter.sheets['Recoupement_Liasses'].set_column('A:A', 50, text_format)
excelwriter.sheets['Recoupement_Liasses'].set_column('B:B', 25, num_format)
excelwriter.sheets['Recoupement_Liasses'].set_column('C:D', 25, text_format)
excelwriter.sheets['Recoupement_Liasses'].set_column('E:E', 60, text_format)

0

In [20]:
# Autofiltre
(max_row, max_col) = balance_df.shape
worksheet_balance.autofilter(0, 0, max_row, max_col - 1)
(max_row, max_col) = financial_df.shape
worksheet_financial.autofilter(0, 0, max_row, max_col - 1)

In [21]:
# Sauvegarde
excelwriter.save()